#
##
##

In [2]:
%load_ext ipydex.displaytools
import pandas as pd
import numpy as np
import seaborn as sns

## 1. (50 points) 
The file flight_delays.csv contains data on more than 2K flights. Each data point contains six features: scheduled departure hour, carrier, destination, origin, weather (i.e., raining, snowing, etc), day of the week, and day of the month. Each feature is measured on a nominal scale. Each data point also contains one response variable: flight status. Apply the Naïve Bayes formula to the entire data set (25 points for correct application) and address the comments and questions given below.

In [12]:
delays_df = pd.read_csv('flight_delays.csv', usecols=range(8))
delays_df.head()
#delays_df.iloc[1]
delays_df.iloc[:,1]

# for each column
# for each distinct 

#for i in delays_df:
#    return 0

0       OH
1       DH
2       DH
3       DH
4       DH
        ..
2196    RU
2197    RU
2198    RU
2199    RU
2200    RU
Name: CARRIER, Length: 2201, dtype: object

### A. (10 points) 
Compute and show the confusion matrix.

### B. (5 points) 
What insights can you obtain from the confusion matrix? (Ensure you are using proper terminology and augmenting the terminology with "clear-text" explanations) 

### C. (5 points)
Suppose your upcoming flight has the following features:
|   |   |
|---|---|
| Scheduled departure hour: | 1600 |
| Carrier: | US |
| Destination: | LGA |
| Origin: | DCA |
| Weather: | yes (i.e.,1) |
| Day of the week:| 6 |
| Day of the month:| 1 |

Using the historical data and the Naïve Bayes formula, do you think your flight will be on time or delayed?

### D. (5 points) 
What is the probability associated with your prediction?

## Problem 2.
(50 points) The file wine.csv contains data on the results of a chemical analysis of 178 wines grown in the same region in Italy but derived from three different cultivars. The analysis determined the quantities of 13 constituents (i.e., features) found in each of the three types of wines. The 13 features are: Alcohol, Malic Acid, Ash, Alcalinity, Magnesium, Total Phenols, Flavanoids, Nonflavoniod Phenols, Proanthocyanins, Color Intensity, Hue, OB280, and Proline. The 14th column is a labeled category stating which of the three cultivars the wine belongs to.   

(20 points for correct application) Apply the k-NN algorithm to the data and address the comments and questions given below.

### A. (10 points) 
Find a good value for k.   Defend your choice using appropriate methodologies. 

### B. (10 points) 
Compute and show the confusion matrix.

### C. (5 points) 
What insights can you obtain from the confusion matrix? (Ensure you are using proper terminology and augmenting the terminology with "clear-text" explanations) 

### D. (5 points) 
Predict the wine cultivar of the following sample point:
Xsample= [11.96   1.63   2.43   17.1   93   2.8   2.74   0.53   1.46   2.6   0.99   3.69   880]